In [8]:
using BenchmarkTools
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [9]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[36])
        V, EV = cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

circle (generic function with 3 methods)

In [10]:
 @btime D,Z= Lar.circle()()

  118.931 μs (1838 allocations: 139.64 KiB)


([1.0 0.98481 … 0.93969 0.98481; 0.0 0.17365 … -0.34202 -0.17365], [[2, 1], [2, 3], [4, 3], [4, 5], [5, 6], [7, 6], [7, 8], [9, 8], [9, 10], [10, 11]  …  [27, 28], [29, 28], [29, 30], [31, 30], [31, 32], [32, 33], [34, 33], [35, 34], [35, 36], [36, 1]])

In [13]:
@code_llvm circle()()


;  @ In[9]:2 within `circle0'
define nonnull %jl_value_t* @julia_circle0_2046([2 x double]* nocapture nonnull readonly dereferenceable(16)) {
top:
  %gcframe = alloca %jl_value_t*, i32 3, align 16
  %1 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %1, i8 0, i32 24, i1 false)
  %thread_ptr = call i8* asm "movq %fs:0, $0", "=r"()
  %ptls_i8 = getelementptr i8, i8* %thread_ptr, i64 -15720
  %ptls = bitcast i8* %ptls_i8 to %jl_value_t***
;  @ In[9]:3 within `circle0'
; ┌ @ array.jl:130 within `vect'
; │┌ @ array.jl:678 within `_array_for'
; ││┌ @ abstractarray.jl:674 within `similar' @ abstractarray.jl:675
; │││┌ @ boot.jl:415 within `Array' @ boot.jl:406
      %2 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
      %3 = bitcast %jl_value_t** %2 to i64*
      store i64 4, i64* %3
      %4 = getelementptr %jl_value_t**, %jl_value_t*** %ptls, i32 0
      %5 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
      %6 = bitcast %j

In [14]:
@code_warntype circle()()

Variables
  #self#::var"#circle0#6"{Float64,Float64}

Body::Tuple{Any,Any}
1 ─ %1 = Base.vect(36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Any,Any}
└──      return %2


In [16]:
@code_typed circle()()

CodeInfo(
1 ─ %1 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Int64,1}, svec(Any, Int64), 0, :(:ccall), Array{Int64,1}, 1, 1))::Array{Int64,1}
│        Base.arraysize(%1, 1)::Int64
└──      goto #3 if not true
2 ─      Base.arrayset(false, %1, 36, 1)::Array{Int64,1}
3 ┄      goto #4
4 ─ %6 = invoke #self#(%1::Array{Int64,1})::Tuple{Any,Any}
└──      return %6
) => Tuple{Any,Any}